In [4]:
import time
import torch
from torch.distributions import Categorical, kl

from net import Net
from aco import ACO
from utils import gen_pyg_data, load_test_dataset

torch.manual_seed(12345)

EPS = 1e-10
device = 'cpu'

In [5]:
def infer_instance(model, instance, n_ants, t_aco_diff):
    distances, adj_mat, prec_cons = instance
    if model:
        model.eval()
        pyg_data = gen_pyg_data(distances, adj_mat, device)
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
        aco = ACO(
            distances=distances,
            prec_cons=prec_cons,
            n_ants=n_ants,
            heuristic=heu_mat,
            device=device
            )
    else:
        aco = ACO(
            distances=distances,
            prec_cons=prec_cons,
            n_ants=n_ants,
            device=device
            )
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t)
        results[i] = best_cost
    return results

@torch.no_grad()
def test(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for instance in dataset:
        results = infer_instance(model, instance, n_ants, t_aco_diff)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on SOP20

DeepACO

In [6]:
n_ants = 20
n_node = 20
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, device)
net = Net().to(device)
net.load_state_dict(torch.load(f'../pretrained/sop/sop{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  20.24020290374756
T=1, average cost is 13.569103240966797.
T=10, average cost is 13.329172134399414.
T=20, average cost is 13.287178039550781.
T=30, average cost is 13.26213264465332.
T=40, average cost is 13.24909782409668.
T=50, average cost is 13.239826202392578.
T=100, average cost is 13.237661361694336.


ACO

In [7]:
n_ants = 20
n_node = 20
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, device)
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  18.865866899490356
T=1, average cost is 15.692425727844238.
T=10, average cost is 14.792289733886719.
T=20, average cost is 14.588752746582031.
T=30, average cost is 14.473760604858398.
T=40, average cost is 14.423331260681152.
T=50, average cost is 14.39449691772461.
T=100, average cost is 14.354658126831055.
